ONT \
This notebook is used to plot the assembly characteristics of the simulation study. ONT reads (mean accuracy = 98%) are simulated by PBSIM3 based on different references. These reads are then used as input for different de novo assembly tools. The metrics of the output assemblies will be plotted here. Metrics to report are:
- Total assembly size
- NG50
- nr of contigs
- nr of errors (found by aligning the assembly back to the reference)

In [ ]:
output_dir = "/home/uitte01p/experimental/test_for_sim_study/simulation_output/ONT_plots"

In [2]:
from matplotlib import pyplot as plt 
import numpy as np 
import bionumpy as bnp
import pandas as pd
from scipy import stats
import seaborn as sns
import os
from Bio import SeqIO

In [3]:
## Nr of contigs
# Plot for ONT for each target and each assembler a figure with the number of contigs on the y axis and the characteristics (readlength, depth) of the reads on the x axis
!find "/home/uitte01p/experimental/test_for_sim_study/simulation_output/ONT" -name *.contigs.fasta > ONT_assembly_filepaths.txt
!find "/home/uitte01p/experimental/test_for_sim_study/simulation_output/ONT" -name *.contigs.stats > ONT_stats_filepaths.txt

In [4]:
with open("ONT_assembly_filepaths.txt") as filepaths:
    all_metrics = []
    for path in filepaths:
        path = path.rstrip()
        fields = path.strip().split('/')[1:] #Split the filepath by /. This will output a '' as first field, remove this by [1:]
        nr_seq = 0
        lengths = []
        IDs = []
        for record in SeqIO.parse(path,"fasta"):
            nr_seq+=1
            length = len(list({record.seq})[0])
            lengths.append(length)
            id = {record.id}
            IDs.append(id)      
        metrics = {
            'technology' : fields[5],
            'assembler' : fields[6],
            'target' : fields[7],
            'accuracy' : fields[8],
            'readlength' : fields[9],
            'depth' : fields[10],
            'number of contigs' : nr_seq,
            'lengths' : lengths,
            'IDs' : IDs,
            'tag' : fields[11].split('.')[0]}
        all_metrics.append(metrics)
    metrics_df = pd.DataFrame(data=all_metrics)

In [6]:
with open("ONT_stats_filepaths.txt") as stats_filepaths:
    all_stats = []
    for stats_path in stats_filepaths:
        stats_path = stats_path.rstrip()
        fields = stats_path.strip().split('/')[1:] #Split the filepath by /. This will output a '' as first field, remove this by [1:]
        !grep ^SN {stats_path} | cut -f 2- > tmp_sn.csv
        df_stats = pd.read_csv("tmp_sn.csv", sep = "\t", header=None)
        bases_mapped = df_stats.iloc[20,1]
        mismatches = df_stats.iloc[23,1]
        error_rate = df_stats.iloc[24,1]
        metrics = {
            'technology' : fields[5],
            'assembler' : fields[6],
            'target' : fields[7],
            'bases mapped' : bases_mapped,
            'mismatches' : mismatches,
            'error rate' : error_rate, 
            'tag' : fields[12].split('.')[0]}
        !rm tmp_sn.csv
        all_stats.append(metrics)
    stats_df = pd.DataFrame(data=all_stats)

In [8]:
def calculate_ng50(contig_lengths, genome_length):
    """
    Calculate the NG50 of a genome assembly.
    
    Parameters:
    contig_lengths (list): List of lengths of contigs.
    genome_length (int): Total length of the genome.
    
    Returns:
    int: NG50 value.
    """
    contig_lengths.sort(reverse=True)
    cumulative_length = 0
    
    for length in contig_lengths:
        cumulative_length += length
        if cumulative_length >= genome_length / 2:
            return length

    return 0  # Return 0 if NG50 cannot be determined

In [9]:
ng50s = []
for lengths_list in metrics_df['lengths']:
    ng50 = calculate_ng50(lengths_list,1000000)
    ng50s.append(ng50)
metrics_df['NG50'] = ng50s

In [10]:
total_lengths = []
for lengths_list in metrics_df['lengths']:
    total_length = sum(lengths_list)
    total_lengths.append(total_length)
metrics_df['total length'] = total_lengths

In [11]:
stats_df

,technology,assembler,target,bases mapped,mismatches,error rate,tag
0,ONT,raven,CHM13_6MB,989462.0,721.0,0.000729,ONT_ac98_rl25000_de30
1,ONT,wtdbg2,CHM13_6MB,991284.0,18.0,0.000018,ONT_ac98_rl25000_de30
2,ONT,flye,CHM13_6MB,995065.0,84.0,0.000084,ONT_ac98_rl25000_de30
3,ONT,canu,CHM13_6MB,1024900.0,9.0,0.000009,ONT_ac98_rl25000_de30
4,ONT,miniasm,CHM13_6MB,787080.0,15033.0,0.019100,ONT_ac98_rl25000_de30


In [12]:
metrics_df

,technology,assembler,target,accuracy,readlength,depth,number of contigs,lengths,IDs,tag,NG50,total length
0,ONT,raven,CHM13_6MB,ac98,rl25000,de30,1,[990328],[{Utg372}],ONT_ac98_rl25000_de30,990328,990328
1,ONT,wtdbg2,CHM13_6MB,ac98,rl25000,de30,1,[992941],[{ctg1}],ONT_ac98_rl25000_de30,992941,992941
2,ONT,flye,CHM13_6MB,ac98,rl25000,de30,1,[995811],[{contig_1}],ONT_ac98_rl25000_de30,995811,995811
3,ONT,canu,CHM13_6MB,ac98,rl25000,de30,2,"[988250, 36672]","[{tig00000001}, {tig00000002}]",ONT_ac98_rl25000_de30,988250,1024922
4,ONT,miniasm,CHM13_6MB,ac98,rl25000,de30,1,[987100],[{utg000001l}],ONT_ac98_rl25000_de30,987100,987100


In [13]:
all_metrics_df = pd.merge(metrics_df,stats_df, how = 'left', on=['assembler','target','tag'])

In [14]:
all_metrics_df

,technology_x,assembler,target,accuracy,readlength,depth,number of contigs,lengths,IDs,tag,NG50,total length,technology_y,bases mapped,mismatches,error rate
0,ONT,raven,CHM13_6MB,ac98,rl25000,de30,1,[990328],[{Utg372}],ONT_ac98_rl25000_de30,990328,990328,ONT,989462.0,721.0,0.000729
1,ONT,wtdbg2,CHM13_6MB,ac98,rl25000,de30,1,[992941],[{ctg1}],ONT_ac98_rl25000_de30,992941,992941,ONT,991284.0,18.0,0.000018
2,ONT,flye,CHM13_6MB,ac98,rl25000,de30,1,[995811],[{contig_1}],ONT_ac98_rl25000_de30,995811,995811,ONT,995065.0,84.0,0.000084
3,ONT,canu,CHM13_6MB,ac98,rl25000,de30,2,"[988250, 36672]","[{tig00000001}, {tig00000002}]",ONT_ac98_rl25000_de30,988250,1024922,ONT,1024900.0,9.0,0.000009
4,ONT,miniasm,CHM13_6MB,ac98,rl25000,de30,1,[987100],[{utg000001l}],ONT_ac98_rl25000_de30,987100,987100,ONT,787080.0,15033.0,0.019100


In [15]:
!mkdir $output_dir
all_metrics_df.to_csv(f"{output_dir}/ONT_assembly_metrics.csv")

mkdir: cannot create directory ‘/home/uitte01p/experimental/test_for_sim_study/simulation_output/ONT_plots’: File exists


In [17]:
#For each assembler, calculate the mean number of contigs for all assemblers
mean_nr_contigs = {'miniasm':np.mean(metrics_df[metrics_df['assembler']=='miniasm']['number of contigs']),
                    'canu':np.mean(metrics_df[metrics_df['assembler']=='canu']['number of contigs']),
                    'flye':np.mean(metrics_df[metrics_df['assembler']=='flye']['number of contigs']),
                    'raven':np.mean(metrics_df[metrics_df['assembler']=='raven']['number of contigs']),
                    'wtdbg2':np.mean(metrics_df[metrics_df['assembler']=='wtdbg2']['number of contigs'])}
sns.scatterplot(mean_nr_contigs)
plt.title("number of contigs per assembler")
plt.savefig(f"{output_dir}/all_overall_mean_nr_contigs.png")
plt.close()

In [ ]:
sns.scatterplot(data=all_metrics_df, x="assembler", y="NG50")
plt.title("NG50 per assembler")
plt.savefig(f"{output_dir}/all_NG50.png")
plt.close()

In [56]:
##canu
canu_metrics = all_metrics_df[all_metrics_df['assembler']=='canu']
canu_metrics_ref = canu_metrics[canu_metrics['target']=="CHM13_6MB"]
canu_metrics_RHD = canu_metrics[canu_metrics['target']=="CHM13_RHDCE"]

##Flye
flye_metrics = all_metrics_df[all_metrics_df['assembler']=='flye']
flye_metrics_ref = flye_metrics[flye_metrics['target']=="CHM13_6MB"]
flye_metrics_RHD = flye_metrics[flye_metrics['target']=="CHM13_RHDCE"]

##miniasm
miniasm_metrics = all_metrics_df[all_metrics_df['assembler']=='miniasm']
miniasm_metrics_ref = miniasm_metrics[miniasm_metrics['target']=="CHM13_6MB"]
miniasm_metrics_RHD = miniasm_metrics[miniasm_metrics['target']=="CHM13_RHDCE"]

##raven
raven_metrics = all_metrics_df[all_metrics_df['assembler']=='raven']
raven_metrics_ref = raven_metrics[raven_metrics['target']=="CHM13_6MB"]
raven_metrics_RHD = raven_metrics[raven_metrics['target']=="CHM13_RHDCE"]

##wtdbg2
wtdbg2_metrics = all_metrics_df[all_metrics_df['assembler']=='wtdbg2']
wtdbg2_metrics_ref = wtdbg2_metrics[wtdbg2_metrics['target']=="CHM13_6MB"]
wtdbg2_metrics_RHD = wtdbg2_metrics[wtdbg2_metrics['target']=="CHM13_RHDCE"]

In [57]:
## Plot the total assembly size for each readlength + depth combination, for each assembler, for each target
for dataset in [canu_metrics_ref,canu_metrics_RHD,flye_metrics_ref,flye_metrics_RHD,miniasm_metrics_ref,miniasm_metrics_RHD,raven_metrics_ref,raven_metrics_RHD,wtdbg2_metrics_ref,wtdbg2_metrics_RHD]:
    assembler = [x for x in globals() if globals()[x] is dataset][0].split('_')[0]
    target = [x for x in globals() if globals()[x] is dataset][0].split('_')[2]
    sns.scatterplot(x=dataset['tag'],y=dataset['total length'])
    plt.title(f"Total length {assembler} assemblies {target}");
    plt.xticks(rotation=90)
    plt.tight_layout()
    plt.savefig(f"{output_dir}/{target}_{assembler}_total_length")
    plt.close()

In [59]:
## Plot the NG50 for each readlength + depth combination, for each assembler, for each target
for dataset in [canu_metrics_ref,canu_metrics_RHD,flye_metrics_ref,flye_metrics_RHD,miniasm_metrics_ref,miniasm_metrics_RHD,raven_metrics_ref,raven_metrics_RHD,wtdbg2_metrics_ref,wtdbg2_metrics_RHD]:
    assembler = [x for x in globals() if globals()[x] is dataset][0].split('_')[0]
    target = [x for x in globals() if globals()[x] is dataset][0].split('_')[2]
    sns.barplot(x=dataset['tag'],y=dataset['NG50'])
    plt.title(f"NG50s {assembler} assemblies {target}");
    plt.xticks(rotation=90)
    plt.ylim(0,1000000)
    plt.tight_layout()
    plt.savefig(f"{output_dir}/{target}_{assembler}_NG50")
    plt.close()

In [58]:
## Plot the number of contigs for each readlength + depth combination, for each assembler, for each target
for dataset in [canu_metrics_ref,canu_metrics_RHD,flye_metrics_ref,flye_metrics_RHD,miniasm_metrics_ref,miniasm_metrics_RHD,raven_metrics_ref,raven_metrics_RHD,wtdbg2_metrics_ref,wtdbg2_metrics_RHD]:
    assembler = [x for x in globals() if globals()[x] is dataset][0].split('_')[0]
    target = [x for x in globals() if globals()[x] is dataset][0].split('_')[2]
    sns.scatterplot(x=dataset['tag'],y=dataset['number of contigs'])
    plt.title(f"Number of contigs {assembler} assemblies {target}");
    plt.xticks(rotation=90)
    plt.tight_layout()
    plt.savefig(f"{output_dir}/{target}_{assembler}_nr_contigs")
    plt.close()

In [60]:
## Plot the number of contigs for each readlength + depth combination, for each assembler, for each target
for dataset in [canu_metrics_ref,canu_metrics_RHD,flye_metrics_ref,flye_metrics_RHD,miniasm_metrics_ref,miniasm_metrics_RHD,raven_metrics_ref,raven_metrics_RHD,wtdbg2_metrics_ref,wtdbg2_metrics_RHD]:
    assembler = [x for x in globals() if globals()[x] is dataset][0].split('_')[0]
    target = [x for x in globals() if globals()[x] is dataset][0].split('_')[2]
    sns.scatterplot(x=dataset['tag'],y=dataset['error rate'])
    plt.title(f"Error rate {assembler} assemblies {target}");
    plt.xticks(rotation=90)
    plt.tight_layout()
    plt.savefig(f"{output_dir}/{target}_{assembler}_error_rate")
    plt.close()

In [ ]:
# !rm assembly_filepaths.txt